# CS6405 - Data Mining - Second Assignment


### Declaration

By submitting this assignment. I agree to the following:

<font color="red">“I have read and understand the UCC academic policy on plagiarism, and agree to the requirements set out thereby in relation to plagiarism and referencing. I confirm that I have referenced and acknowledged properly all sources used in the preparation of this assignment.
I declare that this assignment is entirely my own work based on my personal study. I further declare that I have not engaged the services of another to either assist me in, or complete this assignment”</font>



## Importing Libraries

In [637]:
#importing all the necessary libraries

import os
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA



## Data Preparation

In [615]:
os.getcwd()
# df = pd.read_csv()
# df = pd.read_csv("https://github.com/andvise/DataAnalyticsDatasets/blob/6d5738101d173b97c565f143f945dedb9c42a400/dm_assignment2/sat_dataset_train.csv?raw=true")
original_df = df

In [618]:
#dropping the columns with more than 40% NaN values

#Columns with NaN values along with their sum
df.columns[df.isna().any()].tolist()
df[df.columns[df.isna().any()].tolist()].isna().sum()

#Columns list with NaN values in all rows and more than 70% of the rows
Complete_NaNColumns = ['v_nd_p_weights_entropy','v_nd_n_weights_entropy','c_nd_p_weights_entropy','c_nd_n_weights_entropy', 'band_node_entropy',
                      'band_weights_entropy', 'exo_node_entropy', 'exo_weights_entropy', 'and_node_entropy', 'and_weights_entropy']

df.drop(columns=Complete_NaNColumns, axis = 1, inplace = True)

In [619]:
'''Replacing the NaN values with the mean value in Columns cg_al_node_entropy,
cg_al_weights_entropy,rg_node_entropy,rg_weights_entropy'''

df['cg_al_node_entropy'] = df['cg_al_node_entropy'].fillna(df['cg_al_node_entropy'].mean())
df['cg_al_weights_entropy'] = df['cg_al_weights_entropy'].fillna(df['cg_al_weights_entropy'].mean())
df['rg_node_entropy'] = df['rg_node_entropy'].fillna(df['rg_node_entropy'].mean())
df['rg_weights_entropy'] = df['rg_weights_entropy'].fillna(df['rg_weights_entropy'].mean())
df['big_node_entropy'] = df['big_node_entropy'].fillna(df['big_node_entropy'].mean())
df['big_weights_entropy'] = df['big_weights_entropy'].fillna(df['big_weights_entropy'].mean())



In [620]:
#No columns with NaN values
df.columns[df.isna().any()].tolist()

#Dropping all rows with inf/-inf values 
df = df[np.isfinite(df).all(1)]

#rounding the values to avoid Large values 
df = round(df,100)



In [621]:
#Splitting the predictors and response variable

Y = df['target']
X = df
X.drop("target",axis=1, inplace=True)

# Tasks

## Basic models and evaluation (5 Marks)

Using Scikit-learn, train and evaluate K-NN and decision tree classifiers using 70% of the dataset from training and 30% for testing. For this part of the project, we are not interested in optimising the parameters; we just want to get an idea of the dataset. Compare the results of both classifiers.

In [622]:
#Splitting the data into train and test sets
#Holdout method

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=10, stratify=Y)

In [623]:
#KNN Method

basic_knn = KNeighborsClassifier(n_neighbors=3)

basic_knn.fit(x_train, y_train)

basic_knn_pred = basic_knn.predict(x_test)


In [624]:
#Decision Tree

basic_decision_tree = DecisionTreeClassifier()

basic_decision_tree.fit(x_train, y_train)

basic_decision_tree_pred = basic_decision_tree.predict(x_test)


In [625]:
#KNN
print('Basic Model Evaluations:')
print('=========================')
print('\nKNN Model:\n')
print('Confusion Matrix:')
print(metrics.confusion_matrix(y_test,basic_knn_pred))
print('Accuracy of the model:', round(metrics.accuracy_score(y_test,basic_knn_pred) * 100, 5))

#Decision Tree
print('\nDecision tree Model\n')
print('Confusion Matrix:')
print(metrics.confusion_matrix(y_test,basic_decision_tree_pred))
print('Accuracy of the model:', round(metrics.accuracy_score(y_test,basic_decision_tree_pred) * 100, 5))


Basic Model Evaluations:

KNN Model:

Confusion Matrix:
[[253  25]
 [ 35 158]]
Accuracy of the model: 87.26115

Decision tree Model

Confusion Matrix:
[[277   1]
 [ 11 182]]
Accuracy of the model: 97.45223


## Robust evaluation (10 Marks)

In this section, we are interested in more rigorous techniques by implementing more sophisticated methods, for instance:
* Hold-out and cross-validation.
* Hyper-parameter tuning.
* Feature reduction.
* Feature normalisation.

Your report should provide concrete information of your reasoning; everything should be well-explained.

Do not get stressed if the things you try do not improve the accuracy. The key to geting good marks is to show that you evaluated different methods and that you correctly selected the configuration.

#### 1. Holdout and Cross Validation

In [626]:
#Holdout Cross-validation method
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=10, stratify=Y)
x_train_cv, x_test_cv, y_train_cv, y_test_cv = train_test_split(x_train, y_train, random_state=10, 
                                                                test_size=0.3, stratify=y_train)

In [627]:
#KNN

HoldoutCV_knn = KNeighborsClassifier(n_neighbors=3)

HoldoutCV_knn.fit(x_train_cv, y_train_cv)

HoldoutCV_knn_pred = HoldoutCV_knn.predict(x_test_cv)

print('Accuracy of the KNN model on Holdout-CV data:')
print(round(metrics.accuracy_score(y_test_cv, HoldoutCV_knn_pred) * 100, 5))



HoldoutCV_test_knn_pred = HoldoutCV_knn.predict(x_test)

print('Accuracy of the KNN model on test data:')
print(round(metrics.accuracy_score(y_test, HoldoutCV_test_knn_pred) * 100, 5))


print()
print()

#Decision Tree

HoldoutCV_Dtree = DecisionTreeClassifier()

HoldoutCV_Dtree.fit(x_train_cv, y_train_cv)

HoldoutCV_Dtree_pred = HoldoutCV_Dtree.predict(x_test_cv)

print('Accuracy of the Decision tree model on Holdout-CV data:')
print(round(metrics.accuracy_score(y_test_cv, HoldoutCV_Dtree_pred) * 100, 5))


HoldoutCV_test_Dtree_pred = HoldoutCV_Dtree.predict(x_test)

print('Accuracy of the Decision tree model on test data:')
print(round(metrics.accuracy_score(y_test, HoldoutCV_test_Dtree_pred) * 100, 5))


Accuracy of the KNN model on Holdout-CV data:
84.54545
Accuracy of the KNN model on test data:
84.92569


Accuracy of the Decision tree model on Holdout-CV data:
96.9697
Accuracy of the Decision tree model on test data:
96.60297


#### 2. Hyperparameter tuning

In [628]:
#KNN
knn_grid_params = { 'n_neighbors' : [3,5,7,9,11],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}

knn_tune = KNeighborsClassifier()

#gridsearch for hyperparameters with 10-fold CV
knn_tune_bestModel = GridSearchCV(knn_tune, knn_grid_params, cv=10)

knn_tune_bestModel.fit(x_train, y_train)

print("Best Hyper Parameters:\n", knn_tune_bestModel.best_params_)


knn_tune_bestModel_Pred = knn_tune_bestModel.predict(x_test)

print('\nAccuracy of the KNN model on tuning Hyperparameters:')
print(round(metrics.accuracy_score(y_test, knn_tune_bestModel_Pred) * 100, 5))



Best Hyper Parameters:
 {'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}

Accuracy of the KNN model on tuning Hyperparameters:
88.1104


In [633]:
#Decision Tree

dtree_grid_params = {
    'max_depth': [2, 3, 5, 10],
    'min_samples_leaf': [5, 10, 20],
    'criterion': ["gini", "entropy"], 
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_impurity_decrease' : [0.01, 0.1, 0.2]
}


dtree_tune = DecisionTreeClassifier()

#gridsearch for hyperparameters with 10-fold CV
dtree_tune_bestModel = GridSearchCV(dtree_tune, dtree_grid_params, cv=10)

dtree_tune_bestModel.fit(x_train, y_train)

print("Best Hyper Parameters:\n", dtree_tune_bestModel.best_params_)


dtree_tune_bestModel_Pred = dtree_tune_bestModel.predict(x_test)

print('\nAccuracy of the KNN model on tuning Hyperparameters:')
print(round(metrics.accuracy_score(y_test, dtree_tune_bestModel_Pred) * 100, 5))



Best Hyper Parameters:
 {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'auto', 'min_impurity_decrease': 0.01, 'min_samples_leaf': 10}

Accuracy of the KNN model on tuning Hyperparameters:
97.0276


#### 3. Feature Normalisation

In [629]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=10, stratify=Y)


np.random.seed(42)

x_train_minmax = x_train
x_test_minmax = x_test

#Using min-max normalisation
min_max_scaler = MinMaxScaler()

x_train_minmax = min_max_scaler.fit_transform(x_train_minmax)
x_test_minmax = min_max_scaler.fit_transform(x_test_minmax)


#Using Z-score Normalisation
stnd_scalar = StandardScaler()

x_train_stnd = x_train
x_test_stnd = x_test

x_train_stnd = stnd_scalar.fit_transform(x_train_stnd)
x_test_stnd = stnd_scalar.fit_transform(x_test_stnd)


In [630]:
#KNN Using Feature Normalisation
#min-max Normalisation


knn_min_max = KNeighborsClassifier(n_neighbors=3)

knn_min_max.fit(x_train_minmax, y_train)

knn_min_max_pred = knn_min_max.predict(x_test_minmax)

print('Accuracy of the KNN model using min-max normalisation on features:')
print(round(metrics.accuracy_score(y_test, knn_min_max_pred) * 100, 5))


#Standard Normalisation
knn_stnd = KNeighborsClassifier(n_neighbors=3)

knn_stnd.fit(x_train_stnd, y_train)

knn_stnd_pred = knn_stnd.predict(x_test_stnd)

print('Accuracy of the KNN model using Z-score normalisation on features:')
print(round(metrics.accuracy_score(y_test, knn_stnd_pred) * 100, 5))

print()
print()

#Decision Tree Using Feature Normalisation
#min-max Normalisation


dtree_min_max = DecisionTreeClassifier()

dtree_min_max.fit(x_train_minmax, y_train)

dtree_min_max_pred = dtree_min_max.predict(x_test_minmax)

print('Accuracy of the Decision tree model using min-max normalisation on features:')
print(round(metrics.accuracy_score(y_test, dtree_min_max_pred) * 100, 5))


#Standard Normalisation
dtree_stnd = DecisionTreeClassifier()

dtree_stnd.fit(x_train_stnd, y_train)

dtree_stnd_pred = dtree_stnd.predict(x_test_stnd)

print('Accuracy of the Decision tree model using Z-score normalisation on features:')
print(round(metrics.accuracy_score(y_test, dtree_stnd_pred) * 100, 5))





Accuracy of the KNN model using min-max normalisation on features:
96.60297
Accuracy of the KNN model using Z-score normalisation on features:
96.39066


Accuracy of the Decision tree model using min-max normalisation on features:
76.43312
Accuracy of the Decision tree model using Z-score normalisation on features:
87.47346


#### 4a. Feature Reduction - PCA

In [631]:
#Feature reduction on Z-score normalised data

pca = PCA(n_components = 30)

np.random.seed(42)
x_train_pca = pca.fit_transform(x_train_stnd)
x_test_pca = pca.fit_transform(x_test_stnd)


In [667]:
#KNN (K=3) Using Feature Reduction - PCA

knn_pca = KNeighborsClassifier(n_neighbors=3)

knn_pca.fit(x_train_pca, y_train)

knn_pca_pred = knn_pca.predict(x_test_pca)

print('Accuracy of the KNN model using feature reduction - PCA:')
print(round(metrics.accuracy_score(y_test, knn_pca_pred) * 100, 5))

#Decision tree
#Decision tree Using Feature Reduction - PCA

dtree_pca = DecisionTreeClassifier()

dtree_pca.fit(x_train_pca, y_train)

dtree_pca_pred = dtree_pca.predict(x_test_pca)

print('Accuracy of the Decision tree model using feature reduction - PCA:')
print(round(metrics.accuracy_score(y_test, dtree_pca_pred) * 100, 5))


Accuracy of the KNN model using feature reduction - PCA:
78.55626
Accuracy of the Decision tree model using feature reduction - PCA:
55.62633


#### 4c. Above combined techniques along with the Hyper parameter tuning

In [651]:
stnd_scalar = StandardScaler()

x_train_stnd = x_train
x_test_stnd = x_test

x_train_stnd = stnd_scalar.fit_transform(x_train_stnd)
x_test_stnd = stnd_scalar.fit_transform(x_test_stnd)

#KNN
knn_grid_params = { 'n_neighbors' : [3,5,7,9,11],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}

knn_final_tune = KNeighborsClassifier()

#gridsearch for hyperparameters with 10-fold CV
knn_final_tuning = GridSearchCV(knn_final_tune, knn_grid_params, cv=10)

knn_final_tuning.fit(x_train_stnd, y_train)

print('KNN Model:')
print("Best Hyper Parameters for KNN:\n", knn_final_tuning.best_params_)


knn_final_tuning_pred = knn_final_tuning.predict(x_test_stnd)

print('\nAccuracy of the KNN model on tuning Hyperparameters after pre-processing:')
print(round(metrics.accuracy_score(y_test, knn_final_tuning_pred) * 100, 5))


#Decision Tree

dtree_grid_params = {
    'max_depth': [2, 3, 5, 10],
    'min_samples_leaf': [5, 10, 20],
    'criterion': ["gini", "entropy"], 
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_impurity_decrease' : [0.01, 0.1, 0.2]
}


dtree_final_tuneModel = DecisionTreeClassifier()

#gridsearch for hyperparameters with 10-fold CV
dtree_final_tuning = GridSearchCV(dtree_final_tuneModel, dtree_grid_params, cv=10)

dtree_final_tuning.fit(x_train_stnd, y_train)

print('\n\nDecision Tree Model:')
print("Best Hyper Parameters for Decision tree:\n", dtree_final_tuning.best_params_)


dtree_final_tuning_pred = dtree_final_tuning.predict(x_test_stnd)

print('\nAccuracy of the Decision Tree model on tuning Hyperparameters after pre-processing:')
print(round(metrics.accuracy_score(y_test, dtree_final_tuning_pred) * 100, 5))



KNN Model:
Best Hyper Parameters for KNN:
 {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}

Accuracy of the KNN model on tuning Hyperparameters after pre-processing:
96.60297


Decision Tree Model:
Best Hyper Parameters for Decision tree:
 {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'sqrt', 'min_impurity_decrease': 0.01, 'min_samples_leaf': 5}

Accuracy of the Decision Tree model on tuning Hyperparameters after pre-processing:
91.50743


In [668]:
#REPORT DATAFRAME


report_data = {'Techniques': ['Basic evaluation', 'Hold-out and cross-validation', 'Hold-out and cross-validation Test data',
                              'Feature reduction - PCA', 'Feature Normalisation - minmax', 'Feature Normalisation - Standard',
                             'Hyperparameter tuning', 'Final Model (Hyperparameter on pre-proc. data)'], 
               
               'KNN Model': [87.26115,84.54545,84.92569,78.55626,96.60297,96.39066,88.1104,96.60297],
               'Decision Tree Model': [97.45223,96.9697,96.60297,57.11253,76.43312,87.47346,97.0276,91.50743]}  

report = pd.DataFrame(report_data)


##### PLEASE RUN THE BELOW CELL FOR THE RESULTS DISPLAY

In [669]:
report

,Techniques,KNN Model,Decision Tree Model
0,Basic evaluation,87.26115,97.45223
1,Hold-out and cross-validation,84.54545,96.96970
2,Hold-out and cross-validation Test data,84.92569,96.60297
3,Feature reduction - PCA,78.55626,57.11253
4,Feature Normalisation - minmax,96.60297,76.43312
5,Feature Normalisation - Standard,96.39066,87.47346
6,Hyperparameter tuning,88.11040,97.02760
7,Final Model (Hyperparameter on pre-proc. data),96.60297,91.50743


### Report on Task 2:

In basic evaluation, Holdout Cross validation methods, Decision tree classifier has performed better than the KNN model. We have splitted the data as 70% training and 30% testing data respectively. 

PCA was used as feature reduction technique and for Feature normalisation technique, both the min-max scaling and z-score normalization have been used here.
Accuracy was reduced for decision tree model in the above both techniques, whereas KNN Model got enhanced and perfomed better on after the above techniques were implemented.

In hyperparameter tuning, we found K=7 as optimal parameter for the KNN model and below parameters in case of Decision tree model. Decision tree performed better here and the acccuracy was improved and reached 97.02760%.

Decision tree best parameters- {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'auto', 'min_impurity_decrease': 0.01, 'min_samples_leaf': 10}

In the final model, we have implemented the hyperparameter tuning on the above pre-processed data. For pre-processing the data, PCA was applied on the training data, after which the data was processed using standard normalization technique.
Here, KNN has achieved an highest accuracy of 96.60297%, more than the Decision tree model.

We can conclude from the above results dataframe that the KNN model was a better fit the dataset and achieved a better accuracy after performing the pre-processing of the data and hyperparameter tuning on the model.


## New classifier (10 Marks)

Replicate the previous task for a classifier that we did not cover in class. So different than K-NN and decision trees. Briefly describe your choice.
Try to create the best model for the given dataset.
Save your best model into your github. And create a single code cell that loads it and evaluate it on the following test dataset:
https://github.com/andvise/DataAnalyticsDatasets/blob/main/dm_assignment2/sat_dataset_test.csv

This link currently contains a sample of the training set. The real test set will be released after the submission. I should be able to run the code cell independently, load all the libraries you need as well.

##### RandomForest model is being implemented here as new classifier 

#### 1. Holdout and Cross Validation

In [639]:
##Holdout Cross-validation method
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=10, stratify=Y)
x_train_cv, x_test_cv, y_train_cv, y_test_cv = train_test_split(x_train, y_train, random_state=10, 
                                                                test_size=0.3, stratify=y_train)

randforest_holdout_cv = RandomForestClassifier()

randforest_holdout_cv.fit(x_train_cv, y_train_cv)

randforest_holdout_cv_pred = randforest_holdout_cv.predict(x_test_cv)

print('Accuracy of the Random Forest model on Holdout-CV data:')
print(round(metrics.accuracy_score(y_test_cv, randforest_holdout_cv_pred) * 100, 5))


randforest_holdout_cv_test_pred = randforest_holdout_cv.predict(x_test)

print('Accuracy of the Random Forest model on test data:')
print(round(metrics.accuracy_score(y_test, randforest_holdout_cv_test_pred) * 100, 5))


Accuracy of the Random Forest model on Holdout-CV data:
98.18182
Accuracy of the Random Forest model on test data:
97.66454


#### 2. Feature reduction

In [640]:
#RandomForest Using Feature Reduction - PCA

randforest_pca = RandomForestClassifier()

randforest_pca.fit(x_train_pca, y_train)

randforest_pca_pred = randforest_pca.predict(x_test_pca)

print('Accuracy of the Random Forest model using feature reduction - PCA:')
print(round(metrics.accuracy_score(y_test, randforest_pca_pred) * 100, 5))


Accuracy of the Random Forest model using feature reduction - PCA:
76.85775


#### 3. Feature Normalisation - Peforming Feature reduction on the above normalised data

In [641]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=10, stratify=Y)


#Feature reduction

pca = PCA(n_components = 30)

np.random.seed(42)
x_train_pca_rf = pca.fit_transform(x_train)
x_test_pca_rf = pca.fit_transform(x_test)



#Feature normalisation on the above feature reduction datas
np.random.seed(42)

x_train_minmax_rf = x_train_pca_rf
x_test_minmax_rf = x_test_pca_rf

#Using min-max normalisation
min_max_scaler_rf = MinMaxScaler()

x_train_minmax_rf = min_max_scaler_rf.fit_transform(x_train_minmax_rf)
x_test_minmax_rf = min_max_scaler_rf.fit_transform(x_test_minmax_rf)


#Using Z-score Normalisation
stnd_rf_scalar = StandardScaler()

x_train_stnd_rf = x_train_pca_rf
x_test_stnd_rf = x_test_pca_rf

x_train_stnd_rf = stnd_rf_scalar.fit_transform(x_train_stnd_rf)
x_test_stnd_rf = stnd_rf_scalar.fit_transform(x_test_stnd_rf)


#Features reduction is done using PCA and the dataset is then normalised using both the Minmax normalisation and 
#Z-score (or) Standard normalisation

In [644]:
#RandomForest on above Feature reduced and Feature normalised data


#Min-max normalisation
randforest_minmax_normalised = RandomForestClassifier()
randforest_minmax_normalised.fit(x_train_minmax_rf, y_train)
randforest_minmax_normalised_pred = randforest_minmax_normalised.predict(x_test_minmax_rf)

print('Accuracy of the Random Forest model using PCA and min-max normalisation on features:')
print(round(metrics.accuracy_score(y_test, randforest_minmax_normalised_pred) * 100, 5))


#Standard normalisation
randforest_stand_normalised = RandomForestClassifier()
randforest_stand_normalised.fit(x_train_stnd_rf, y_train)
randforest_stand_normalised_pred = randforest_stand_normalised.predict(x_test_stnd_rf)

print('Accuracy of the Random Forest model using PCA and Standard normalisation on features:')
print(round(metrics.accuracy_score(y_test, randforest_stand_normalised_pred) * 100, 5))



Accuracy of the Random Forest model using PCA and min-max normalisation on features:
60.08493
Accuracy of the Random Forest model using PCA and Standard normalisation on features:
80.67941


#### 4. Hyperparameter tuning on above

In [670]:
rf_param_grid = {'criterion':['gini','entropy'],
          'n_estimators':[25,30,40,50],
          'max_depth': [100, 500],
          'min_samples_leaf':[1,2,3],
          'min_samples_split':[5,7,9,11,13,15], 
          
}

rf_grid_search = RandomForestClassifier()

rf_hyperparam = GridSearchCV(rf_grid_search, param_grid = rf_param_grid, cv = 10)

rf_hyperparam.fit(x_train_stnd_rf,y_train)

print('Best parameters for this model are: ', rf_hyperparam.best_params_)
rf_hyperparam_pred = rf_hyperparam.predict(x_test_stnd_rf)
print('\nAccuracy of the Random Forest model after tuning the hyperparameters on the above pre-processed data:')
print(round(metrics.accuracy_score(y_test, rf_hyperparam_pred) * 100, 5))


Best parameters for this model are:  {'criterion': 'entropy', 'max_depth': 500, 'min_samples_leaf': 3, 'min_samples_split': 5, 'n_estimators': 25}

Accuracy of the Random Forest model after tuning the hyperparameters on the above pre-processed data:
77.49469


#### Final Model results - Random Forest classifier
Best parameters for this model are:  {'criterion': 'entropy', 'max_depth': 100, 'min_samples_leaf': 1, 'min_samples_split': 7, 'n_estimators': 25}

Accuracy of the Random Forest model after tuning the hyperparameters on the above pre-processed data:
80.04246

In [677]:
#REPORT DATAFRAME


report_data_rf = {'Techniques': ['Hold-out and cross-validation', 'Hold-out and cross-validation Test data',
                              'Feature reduction - PCA', 'Feature Normalisation - minmax on PCA ', 'Feature Normalisation - Standard on PCA',
                             'Final Model (Hyperparameter on pre-proc. data)'], 
               
               'Random Forest Classifer': [98.18182,97.66454,76.85775,60.08493,80.67941,77.49469]}  

report_rf = pd.DataFrame(report_data_rf)


##### PLEASE RUN THE BELOW CELL FOR THE RESULTS DISPLAY

In [678]:
report_rf

,Techniques,Random Forest Classifer
0,Hold-out and cross-validation,98.18182
1,Hold-out and cross-validation Test data,97.66454
2,Feature reduction - PCA,76.85775
3,Feature Normalisation - minmax on PCA,60.08493
4,Feature Normalisation - Standard on PCA,80.67941
5,Final Model (Hyperparameter on pre-proc. data),77.49469


### Report on Task 3

It is clearly evident from the results table the Random forest classifier performed better on the holdout crossvalidation method with an highest accuracy of 98.18%. While the accuracy was reduced further when the above techniques were applied on the data as well on the model, the final model accuracy was increasing with the best parameter search.

It is evident that the model will perform better with more rigorous gridsearch of hyperparameters. Due to local resource crunch (CPU, RAM, GPU etc), only fewer parameters were involved in model search. When the best set of parameters is found, the model accuracy will improve and it will predict the test data with higher accuracy.

# <font color="blue">FOR GRADING ONLY</font>

Save your best model into your github. And create a single code cell that loads it and evaluate it on the following test dataset: 
https://github.com/andvise/DataAnalyticsDatasets/blob/main/dm_assignment2/sat_dataset_test.csv

In [ ]:
from joblib import dump, load
from io import BytesIO
import requests

# INSERT YOUR MODEL'S URL
mLink = 'URL_OF_YOUR_MODEL_SAVED_IN_YOUR_GITHUB_REPOSITORY?raw=true'
mfile = BytesIO(requests.get(mLink).content)
model = load(mfile)
# YOUR CODE HERE